There is are no errors in the forecasts for weather conditions in citylearn. THis notbook calculates the mean error introduced by the ACG attack.

In [1]:
import numpy as np
import pandas as pd

The observations contain current and forecast measurements, we will analyze how the attack affects the accuracy of forecasts

In [2]:
df_acg = pd.read_csv('adv_perturbed_obs.csv', index_col=0)


Only keep weather/pricing data

In [3]:
df_acg = df_acg[df_acg.columns[df_acg.columns.str.contains(f'temp|humidity|solar|pricing')]]

Align forcasts with measurements

In [4]:
forecasts = [6,12,24]
for i in forecasts:
     df_acg[df_acg.columns[df_acg.columns.str.contains(f'{i}')]] =\
          df_acg.loc[:,df_acg.columns[df_acg.columns.str.contains(f'{i}')]].shift(i)

Calculated the differenc between the newly aligned forecasts and the acutal measurements

In [5]:
for measurement in df_acg.loc[:,~df_acg.columns.str.contains('predicted')].columns: #column names which don't contain predicted
    df_acg[df_acg.columns[df_acg.columns.str.contains(f'{measurement}_predicted')]] = \
        df_acg[df_acg.columns[df_acg.columns.str.contains(f'{measurement}_predicted')]].sub(df_acg[measurement], axis='rows').abs() #distance between the measurement and prediction

Note that if the abs is ommitted above, the difference is ~0.001% or less (you can just remove the abs above, this is way faster than recoding everything)

rename the prediction columns to indicate they now contain an error value

In [6]:
df_acg.rename(columns=lambda name: name + '_error' if 'predicted' in name else name, inplace=True)

Compute the mean and it's standard error (SEM):

In [7]:
df_acg[df_acg.columns[df_acg.columns.str.contains('error')]].agg(['mean','sem']).transpose()

,mean,sem
outdoor_dry_bulb_temperature_predicted_6h_error,0.028053,0.000233
outdoor_dry_bulb_temperature_predicted_12h_error,0.028113,0.000234
outdoor_dry_bulb_temperature_predicted_24h_error,0.027250,0.000235
outdoor_relative_humidity_predicted_6h_error,0.027598,0.000231
outdoor_relative_humidity_predicted_12h_error,0.027418,0.000229
outdoor_relative_humidity_predicted_24h_error,0.026976,0.000235
diffuse_solar_irradiance_predicted_6h_error,0.028253,0.000237
diffuse_solar_irradiance_predicted_12h_error,0.028215,0.000234
diffuse_solar_irradiance_predicted_24h_error,0.027628,0.000238
direct_solar_irradiance_predicted_6h_error,0.028491,0.000240


We see that the absoulte prediction error is increased by ~2.7% by the ACG attack, which is slightly higher that the mean epsilon value below

In [10]:
s_eps = pd.read_csv('Epsilons.csv', index_col=0)
s_eps.agg(['mean','sem'])

,0
mean,0.023561
sem,0.000159
